In [128]:
import re
import pandas as pd # CSV file I/O (pd.read_csv)
from nltk.corpus import stopwords

import copy
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import VarianceThreshold
from imblearn.over_sampling import SMOTE
from sklearn.dummy import DummyClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import accuracy_score
#from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [129]:
news = pd.read_excel("C:\\Users\\Kratika\\Downloads\\News data (Completed) (1) (9).xlsx")
news.head()

,Title,Body,Category
0,Adani Ports Shares Jump Nearly 6% as Buyback O...,"August 26, 2019, 4:25 PM IST\nFor Representati...",Buyback of shares - Alert me when any of these...
1,Adani Ports buyback to open on September 6; sh...,Adani Ports and Special Economic Zone\ngained ...,Buyback of shares - Alert me when any of these...
2,Adani Ports Q1 net profit jumps 47.50%,Adani Ports and Special Economic Zone (APSEZ) ...,"Earnings, Stock Performance or Dividends - Ale..."
3,Adani Ports keen on building port at Nagarjuna...,Adani Ports and Special Economic Zone Ltd (APS...,New Geography - Alert me when a company moves ...
4,Adani Ports Q1 result: Adani Ports Q1 PAT jump...,Adani Ports\non Wednesday said its consolidate...,"Earnings, Stock Performance or Dividends - Ale..."


In [131]:
from nltk.tokenize import word_tokenize 

def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    stop_words.union(['january','february','march','april','may','june','july','august','september','october','november','december'])
    stop_words.union(['jan','feb','mar','apr','may','jun','jul','aug','sept','oct','nov','dec'])
    stop_words.union(['monday', 'tuesday', 'wednesday', 'thursday','friday','saturday','sunday'])
    word_tokens = word_tokenize(data) 
    filtered_sentence = []
    
    for w in word_tokens: 
        if w.lower() not in stop_words: 
            filtered_sentence.append(w) 
    
    return " ".join(filtered_sentence)

In [132]:
news['clean_body'] = news['Body'].apply(remove_stop_words) 
news['clean_title'] = news['Title'].apply(remove_stop_words) 
#remove_stop_words_Body

In [169]:
# ## Split data
print("\nSplitting data")
title_tr, title_te, category_tr, category_te = train_test_split(news['clean_title'], news.Category,test_size =.02)
title_tr, title_de, category_tr, category_de = train_test_split(title_tr,category_tr,test_size =.02)
print("Training: ",len(title_tr))
print("Developement: ",len(title_de),)
print("Testing: ",len(title_te))


Splitting data
Training:  2398
Developement:  49
Testing:  50


In [170]:
#from sklearn.preprocessing import LabelEncoder

# # Data Preprocessing
# ## Vectorization of data
# Vectorize the data using Bag of words (BOW)
print("\nVectorizing data")
#tokenizer = nltk.tokenize.RegexpTokenizer(r"\w+")
#stop_words = nltk.corpus.stopwords.words("english")
#vectorizer = CountVectorizer(tokenizer=tokenizer.tokenize, stop_words=stop_words)

#vectorizer.fit(iter(title_tr))
#Xtr = vectorizer.transform(iter(title_tr))
#Xde = vectorizer.transform(iter(title_de))
#Xte = vectorizer.transform(iter(title_te))

#encoder = LabelEncoder()
#encoder.fit(category_tr)
#Ytr = encoder.transform(category_tr)
#Yde = encoder.transform(category_de)
#Yte = encoder.transform(category_te)


{'TCS': 3538,
 'Wipro': 3900,
 '000': 1,
 'Honchos': 1826,
 'Forced': 1593,
 'Give': 1692,
 'Stakes': 3434,
 'Govt': 1714,
 'Orders': 2680,
 'Shares': 3310,
 'Public': 2926,
 'Coal': 1018,
 'India': 1930,
 'could': 4476,
 'close': 4381,
 '53': 241,
 'underground': 6351,
 'mines': 5362,
 'fiscal': 4869,
 'SBI': 3173,
 'cuts': 4520,
 'loan': 5267,
 'deposit': 4577,
 'prices': 5654,
 'ultratech': 6343,
 'cement': 4324,
 'Buy': 830,
 'Ultratech': 3739,
 'Cement': 930,
 'target': 6221,
 'Rs': 3150,
 '247': 118,
 'Axis': 621,
 'Securities': 3259,
 'Bajaj': 670,
 'Finserv': 1564,
 'Merck': 2398,
 'Bandhan': 673,
 'Bank': 677,
 'likely': 5239,
 'limelight': 5240,
 'today': 6275,
 'Kampala': 2091,
 'based': 4156,
 'drugs': 4664,
 'firm': 4866,
 'Cipla': 994,
 'raises': 5730,
 'Sh4': 3290,
 'billion': 4195,
 'IPO': 1894,
 'Kenya': 2108,
 'Mahindra': 2312,
 'Marazzo': 2352,
 'Price': 2882,
 'Specification': 3411,
 'New': 2571,
 'MPV': 2298,
 'Car': 907,
 'Stocks': 3466,
 'news': 5434,
 'HCL': 174

In [171]:
from keras.models import Sequential

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [151]:
Xtr

<2398x5389 sparse matrix of type '<class 'numpy.int64'>'
	with 20247 stored elements in Compressed Sparse Row format>

In [152]:
# ## Feature Reduction
# We can check the variance of the feature and drop them based on a threshold
print("\nApplyting Feature Reduction")
print("Number of features before reduction : ", Xtr.shape[1])
selection = VarianceThreshold(threshold=0.001)
Xtr_whole = copy.deepcopy(Xtr)
Ytr_whole = copy.deepcopy(Ytr)
selection.fit(Xtr)
Xtr = selection.transform(Xtr)
Xde = selection.transform(Xde)
Xte = selection.transform(Xte)
print("Number of features after reduction : ", Xtr.shape[1])


Applyting Feature Reduction
Number of features before reduction :  5389
Number of features after reduction :  1467


In [161]:
# # Train Models
# ### Baseline Model
# “stratified”: generates predictions by respecting the training set’s class distribution.
print("\n\nTraining baseline classifier")
dc = DummyClassifier(strategy="stratified")
dc.fit(title_tr, category_tr)
pred = dc.predict(title_de)



Training baseline classifier


In [162]:
print(classification_report(category_de, pred))

                                                                                                                                     precision    recall  f1-score   support

                                                                                 Annual Report - Comments made in the Annual Report       0.00      0.00      0.00         0
                                                     Buyback of shares - Alert me when any of these companies buys back it's shares       0.00      0.00      0.00         1
                                    Debt Financing-Alert me when any of these companies receives a loan they have promised to repay       0.00      0.00      0.00         1
               Earnings, Stock Performance or Dividends - Alert me when any of these companies release numbers on their performance       0.00      0.00      0.00         5
                            Executive Quote or Publication - Alert me when any of these companies executives is quoted or published   

In [139]:

# ### Decision Tree
print("Training Decision tree")
dt = DecisionTreeClassifier()
dt.fit(Xtr, Ytr)
pred = dt.predict(Xde)
print(classification_report(Yde, pred))

Training Decision tree
              precision    recall  f1-score   support

           3       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         3
          16       1.00      1.00      1.00         1
          20       0.00      0.00      0.00         1
          26       0.00      0.00      0.00         1
          27       0.00      0.00      0.00         1
          29       0.00      0.00      0.00         1
          31       0.00      0.00      0.00         1
          32       0.00      0.00      0.00         2
          34       1.00      0.50      0.67         2
          35       0.00      0.00      0.00         1
          37       0.33      0.50      0.40         2
          39       0.00      0.00      0.00         0
          40       0.00      0.00      0.00         2
    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [155]:
# ### Random Forest
print("Training Random Forest")
rf = RandomForestClassifier(n_estimators=40)
rf.fit(Xtr, Ytr)
pred = rf.predict(Xde)
print(classification_report(Yde, pred))

Training Random Forest
              precision    recall  f1-score   support

           3       1.00      1.00      1.00         1
           4       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
          10       0.75      1.00      0.86         6
          20       0.00      0.00      0.00         2
          24       1.00      1.00      1.00         2
          31       0.00      0.00      0.00         0
          32       0.75      1.00      0.86         3
          34       1.00      1.00      1.00         1
          37       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         1
          40       0.00      0.00      0.00         2
          42       0.00      0.00      0.00         2
          46       0.00      0.00      0.00         1
          47       1.00      0.50      0.67         2
          49       1.00      1.00      1.00         1
          50       0.00      0.00      0.00         1
    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [156]:

# ### Multinomial Naive Bayesian
print("Training Multinomial Naive Bayesian")
nb = MultinomialNB()
nb.fit(Xtr, Ytr)
pred_nb = nb.predict(Xde)
print(classification_report(Yde, pred_nb))


Training Multinomial Naive Bayesian
              precision    recall  f1-score   support

           3       1.00      1.00      1.00         1
           4       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
          10       0.60      1.00      0.75         6
          20       0.00      0.00      0.00         2
          24       1.00      1.00      1.00         2
          31       0.00      0.00      0.00         0
          32       0.67      0.67      0.67         3
          34       0.50      1.00      0.67         1
          37       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         1
          40       0.00      0.00      0.00         2
          42       0.00      0.00      0.00         2
          46       0.00      0.00      0.00         1
          47       1.00      0.50      0.67         2
          49       0.00      0.00      0.00         1
          50       0.00      0.00      0.00  

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [157]:
# ### Support Vector Classification
print("Training Support Vector Classification")
from sklearn.svm import SVC
svc = SVC()
svc.fit(Xtr, Ytr)
pred = svc.predict(Xde)
print(classification_report(Yde, pred))


Training Support Vector Classification


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


              precision    recall  f1-score   support

           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         1
           7       0.00      0.00      0.00         1
          10       0.00      0.00      0.00         6
          20       0.00      0.00      0.00         2
          24       0.00      0.00      0.00         2
          32       0.00      0.00      0.00         3
          34       0.00      0.00      0.00         1
          37       0.00      0.00      0.00         1
          39       0.00      0.00      0.00         1
          40       0.00      0.00      0.00         2
          42       0.00      0.00      0.00         2
          46       0.00      0.00      0.00         1
          47       0.00      0.00      0.00         2
          49       0.00      0.00      0.00         1
          50       0.00      0.00      0.00         1
          59       0.37      1.00      0.54        18
          60       0.00    

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [163]:
# ### Multilayered Perceptron
print("Training Multilayered Perceptron")
mlp = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(100, 20), random_state=1, max_iter=400)
mlp.fit(title_tr, category_tr)
pred = mlp.predict(title_de)
print(classification_report(category_de, pred))

Training Multilayered Perceptron


ValueError: could not convert string to float: 'Bharti Infratel Towers Competition'

In [159]:
# # Final Model: Multinomial Naive Bayesian
# **Multinomial Naive Bayesian** works the best. Lets run NB on our test data and get the confusion matrix and its heat map.
# ## Predict test data
print("\n\nPredicting test data using Multinomial Naive Bayesian")
pred_final = nb.predict(Xte)
print(classification_report(Yte, pred_final))




Predicting test data using Multinomial Naive Bayesian
              precision    recall  f1-score   support

           3       1.00      1.00      1.00         1
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         2
          10       0.56      0.83      0.67         6
          11       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         1
          20       0.00      0.00      0.00         2
          24       1.00      1.00      1.00         1
          32       0.00      0.00      0.00         1
          34       1.00      1.00      1.00         2
          39       1.00      1.00      1.00         1
          42       0.00      0.00      0.00         1
          44       0.00      0.00      0.00         1
          49       1.00      1.00      1.00         1
          50       0.00  